# Computer Vision Model Deployment


# Introduction:

Operationalization is the process of publishing models and code as web services and the consumption of these services to produce business results. Once your model is trained, you can deploy your trained model as a webservice for consumption with [Azure Machine Learning CLI](https://docs.microsoft.com/en-us/azure/machine-learning/preview/cli-for-azure-machine-learning). The API can automatically generate required information and call [Azure Machine Learning Model Management](https://docs.microsoft.com/en-us/azure/machine-learning/desktop-workbench/model-management-overview) service to deploy your model. Your model can be deployed to an Azure Container Service (ACS) cluster as a webservice. It also provides some autoscaling functionality for your webservice. Also provided are instructions on how to deploy your model to edge devices to apply computer vision models to data that can’t make it to the cloud due to data sovereignty, privacy, and/or bandwidth issues.

What is covered in this notebook: 
1. <a href='#prerequisite'>Prerequisites</a>
- <a href='#cloud_deployment'>Cloud Deployment</a>
- <a href='#webservice_consumption'>Webservice Consumption</a>
- <a href='#edge_deployment'>Edge Deployment</a>
- <a href='#FAQ'>Advanced topics and FAQ</a>
- <a href='#troubleshooting'>Troubleshooting</a>

# 1. Prerequisites:<a id='prerequisite'></a>
   - You will need the following required CLI setup ([Azure](https://azure.microsoft.com/en-us/) account, [Azure ML Model Management](https://docs.microsoft.com/en-us/azure/machine-learning/preview/model-management-overview) account, deployment environment) before running the deployment code. You only need to set it up once for all your deployments. **Note: Please DO NOT use Azure ML Model Management account with region of "eastus2" now. Deployment environment (ACS cluster) with region of "westcentralus" has known provision issue.**

   - You need an [Azure](https://azure.microsoft.com/en-us/) account with a valid subscription. You need to login to your account if you haven't done so. Change to your target subscription if you need.
   >Azure CLI command to login: 
   `az login` 
   
   >Azure CLI command to change subscription: 
   `az account set --subscription [your subscription name]` 
   
   - You need an [Azure ML Model Management](https://docs.microsoft.com/en-us/azure/machine-learning/preview/model-management-overview) account. You need to set your model management account in CLI if you haven't done it before. You only need to set it once for all your deployments. **Note: Please create your Azure Machine Learning Model Management account with location close to your location, avoiding "eastus2" for now (Because some locations are having deployment timeout issue).** For more details, you can follow the instructions [here](https://docs.microsoft.com/en-us/azure/machine-learning/preview/deployment-setup-configuration#create-a-model-management-account). The below CLI command shows how to create your active model management account: `az ml account modelmanagement show`
   >Azure CLI command example to create and set model management account:
   ```
   az ml account modelmanagement create -l [Azure region, e.g. westcentralus] -n [your account name] -g [resource group name] --sku-instances [number of instances, e.g. 1] --sku-name [Pricing tier for example S1]
   az ml account modelmanagement set -n [your account name] -g [resource group it was created in]
   ``` 
   - You need a deployment environment (cluster). If you don't have one, please follow the CLI command example provided below to set up a cloud deployment environment. More details can be found in this [page](https://docs.microsoft.com/en-us/azure/machine-learning/preview/deployment-setup-configuration#environment-setup). **Note: Deployment environment (ACS cluster) with region of "westcentralus" has known provision issue.** You can use this CLI command to show your active deployment environment: ```az ml env show```
   
   >Azure CLI command example to create and set deployment environment
    ```
    az provider register -n Microsoft.MachineLearningCompute
    az provider register -n Microsoft.ContainerRegistry
    az provider register -n Microsoft.ContainerService
    az ml env setup --cluster -n [your environment name] -l [Azure region e.g. eastus2] [-g [resource group]]
    az ml env set -n [environment name] -g [resource group]
    az ml env cluster
    ```

In [ ]:
# ##### OPTIONAL - Interactive CLI setup helper ###### 
# # Interactive CLI setup helper, including model management account and deployment environment.
# # If you haven't setup you CLI before or if you want to change you CLI settings, you can use this block to help you interactively.
# # UNCOMMENT THE FOLLOWING LINES IF YOU HAVE NOT CREATED OR SET THE MODEL MANAGEMENT ACCOUNT AND DEPLOYMENT ENVIRONMENT

# from azuremltkbase.deployment import CliSetup
# CliSetup().run()

# 2. Cloud Deployment <a id='cloud_deployment'></a>

In [ ]:
# import modules
import warnings
warnings.filterwarnings("ignore")
import json, numpy as np, os, timeit 
from IPython.display import display
from cvtk import ClassificationDataset, CNTKTLModel, Context, Splitter, StorageContext
import cvtk
import matplotlib.pyplot as plt
%matplotlib inline

# Disable printing of logging messages
from azuremltkbase.logging import ToolkitLogger
ToolkitLogger.getInstance().setEnabled(False)

# Set storage context.
out_root_path = "../../../cvtk_output"
Context.create(outputs_path=out_root_path, persistent_path=out_root_path, temp_path=out_root_path)

### Load a saved model for deployment
You can persist your model on disk and reuse it later for deployment. After that you can load a previously saved model for deployment. Note: Object detection model currently doesn't support: saving to disk and reusing it for deployment. However you can directly use your object detection object for deployment.

> You use the following example code to save your model to disk and reuse it for deployment.

```
import os
save_model_path = os.path.join(Context.get_global_context().storage.persistent_path, "saved_classifier.model")
# Save model to disk
dnn_model.save(save_model_path)
```

In [ ]:
# # Example code to load a saved model from disk
# from cvtk import CNTKTLModel, Context
# dnn_model = CNTKTLModel.load(save_model_path)

### Deploy a computer vision model
First, create an AMLDeployment object (`deploy_obj` in the example) by providing a name (Please use lower case alphabets and numeric format with lenth 3-32. Example: mydeployment3), your model and other optional information. Then you can use `deploy()` function to deploy your model. Once the deployment is finished. You will get your service id, service endpoint url, and service key. Please keep them for future reference.

In [ ]:
from cvtk.operationalization import AMLDeployment

# Set deployment name
# Please use lower case alphabets and numeric format with lenth 3-32. Example: mydeployment3
deployment_name = "wsdeployment"

# Create deployment object
# It will use the current deployment environment (you can check it with CLI command "az ml env show").
deploy_obj = AMLDeployment(deployment_name=deployment_name, aml_env="cluster", associated_DNNModel=dnn_model, replicas=1)

# Alternatively, you can provide azure machine learning deployment cluster name (environment name) and resource group name
# to deploy your model. It will use the provided cluster to deploy. To do that, please uncomment the following lines to create 
# the deployment object.

# azureml_rscgroup = "<resource group>"
# cluster_name = "<cluster name>"
# deploy_obj = AMLDeployment(deployment_name=deployment_name, associated_DNNModel=dnn_model,
#                            aml_env="cluster", cluster_name=cluster_name, resource_group=azureml_rscgroup, replicas=1)

# Check if the deployment name exists, if yes remove it first.
# Note: This will delete existing webservice with the same name. Do not delete the webservice unintentionally. 
if deploy_obj.is_existing_service():
    AMLDeployment.delete_if_service_exist(deployment_name)
    
# create the webservice
print("Deploying to Azure cluster...")
deploy_obj.deploy()
print("Deployment DONE")

# 3. Webservice comsumption

Once you created the webservice, you can score images with the deployed webservice. You have several options:

   - Directly score the webservice with the deployment object: deploy_obj.score_image(image_path_or_url) 
   - Service endpoint URL and Serivce key (none for local deployment): AMLDeployment.score_existing_service_with_image(image_path_or_url, service_endpoint_url, service_key=None)
   - HTTP requests directly to score the webservice endpoint (for advanced users).

#### Score with existing deployment object
```
deploy_obj.score_image(image_path_or_url)
```

In [ ]:
# Score with existing deployment object

# Score local image with file path
print("Score local image with file path")
image_path_or_url = test_set.images[0].storage_path
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url, image_resize_dims=[224,224])
print("serialized_result_in_json:", serialized_result_in_json)

# Score image url and remove image resizing
print("Score image url")
image_path_or_url = "https://cvtkdata.blob.core.windows.net/publicimages/microsoft_logo.jpg"
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url)
print("serialized_result_in_json:", serialized_result_in_json)

# Score image url with added paramters. Add softmax to score.
print("Score image url with added paramters. Add softmax to score")
from cvtk.utils.constants import ClassificationRESTApiParamters
image_path_or_url = "https://cvtkdata.blob.core.windows.net/publicimages/microsoft_logo.jpg"
print("Image source:",image_path_or_url)
serialized_result_in_json = deploy_obj.score_image(image_path_or_url, image_resize_dims=[224,224], parameters={ClassificationRESTApiParamters.ADD_SOFTMAX:True})
print("serialized_result_in_json:", serialized_result_in_json)

In [ ]:
# Time image scoring
import timeit

num_images = 10
for img_index, img_obj in enumerate(test_set.images[:num_images]):
    print("Calling API for image {} of {}: {}...".format(img_index, num_images, img_obj.name))
    tic = timeit.default_timer()
    return_json = deploy_obj.score_image(img_obj.storage_path, image_resize_dims=[224,224])
    print("   Time for API call: {:.2f} seconds".format(timeit.default_timer() - tic))
    print(return_json)

#### Score with service endpoint url and service key
`AMLDeployment.score_existing_service_with_image(image_path_or_url, service_endpoint_url, service_key=None)`

In [ ]:
# Import related classes and functions
from cvtk.operationalization import AMLDeployment

service_endpoint_url = "" # please replace with your own service url
service_key = "" # please replace with your own service key
# score local image with file path
image_path_or_url = test_set.images[0].storage_path
print("Image source:",image_path_or_url)
serialized_result_in_json = AMLDeployment.score_existing_service_with_image(image_path_or_url,service_endpoint_url, service_key = service_key)
print("serialized_result_in_json:", serialized_result_in_json)

# score image url
image_path_or_url = "https://cvtkdata.blob.core.windows.net/publicimages/microsoft_logo.jpg"
print("Image source:",image_path_or_url)
serialized_result_in_json = AMLDeployment.score_existing_service_with_image(image_path_or_url,service_endpoint_url, service_key = service_key, image_resize_dims=[224,224])
print("serialized_result_in_json:", serialized_result_in_json)

#### Score endpoint with http request directly
Following is some example code to form the http request directly in Python. This can be done in other programming languages since it uses a direct http request.

In [ ]:
def score_image_with_http(image, service_endpoint_url, service_key=None, parameters={}):
    """Score local image with http request

    Args:
        image (str): Image file path
        service_endpoint_url(str): web service endpoint url
        service_key(str): Service key. None for local deployment.
        parameters (dict): Additional request paramters in dictionary. Default is {}.


    Returns:
        str: serialized result 
    """
    import requests
    from io import BytesIO
    import base64
    import json

    if service_key is None:
        headers = {'Content-Type': 'application/json'}
    else:
        headers = {'Content-Type': 'application/json',
                   "Authorization": ('Bearer ' + service_key)}
    payload = []
    encoded = None
    
    # Read image
    with open(image,'rb') as f:
        image_buffer = BytesIO(f.read()) ## Getting an image file represented as a BytesIO object
        
    # Convert your image to base64 string
    # image_in_base64 : "b'{base64}'"
    encoded = base64.b64encode(image_buffer.getvalue())
    image_request = {"image_in_base64": "{0}".format(encoded), "parameters": parameters}
    payload.append(image_request)
    body = json.dumps(payload)
    r = requests.post(service_endpoint_url, data=body, headers=headers)
    try:
        result = json.loads(r.text)
        json.loads(result[0])
    except:
        raise ValueError("Incorrect output format. Result cant not be parsed: " + r.text)
    return result[0]


### Parse serialized result from webservice
The result from the webserice is a json formated string. You can follow the example code in each notebooks to parse it (**Parse serialized result from webservice** section).

# 4. Edge Deployment<a id='edge_deployment'></a>

## Introduction:

The integration of computer vison models and [Azure IoT Edge](https://aka.ms/azure-iot-edge-doc) enables organizations and developers to apply computer vision models to data that can’t make it to the cloud due to data sovereignty, privacy, and/or bandwidth issues. All models created using the AML package can now be deployed to IoT gateways and devices with the Azure IoT Edge runtime. Models are operationalized as containers by [Azure Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/preview/) and can run on many types of hardware.

Azure IoT Edge moves cloud analytics and custom business logic to devices so that your organization can focus on business insights instead of data management. Enable your solution to truly scale by configuring your IoT software, deploying it to devices via standard containers, and monitoring it all from the cloud.

This notebook provides instructions on how to deploy computer vision models developed by the AML package in Azure IoT Edge-compatible Docker containers and expose those models as REST APIs.

###  Prerequisites 
- Edge device with supported platform - Windows 10, Linux/Mac. Note: Windows DSVM/DLVM (Windows Server OS) is not supported well now. If you can get docker running in Linux container mode in your Windows Server device, it also works (Out of scope of this instruction). 
- Docker. Make sure docker is running on your device, and in Linux containers mode. If you don't have it on your device, please install it first: 
    * [Install Docker for Windows](https://docs.docker.com/docker-for-windows/install/)
    * [Install Docker for Linux](https://docs.docker.com/install/linux/docker-ce/ubuntu/)
    * [Install Docker for Mac](https://docs.docker.com/docker-for-mac/install/)
- Python pip, make sure you can use the pip command. 
   * [Install Python on Windows](https://www.python.org/downloads/)
   * MacOS: `sudo easy_install pip`
   * Lixun: `sudo apt-get install python-pip`
- IoT Edge runtime:
    * `pip install -U azure-iot-edge-runtime-ctl`


## Setup:
1. [Create an IoT hub with Azure CLI](https://docs.microsoft.com/en-us/azure/iot-edge/quickstart#create-an-iot-hub-with-azure-cli), if you don't have one.
1. [Register an IoT Edge device](https://docs.microsoft.com/en-us/azure/iot-edge/quickstart#register-an-iot-edge-device)
1. Configure the IoT Edge runtime. [Windows](https://docs.microsoft.com/en-us/azure/iot-edge/quickstart#configure-the-iot-edge-runtime), [Linux/Mac](https://docs.microsoft.com/en-us/azure/iot-edge/quickstart#configure-the-iot-edge-runtime)
- Create the Docker container with the package:
    * Train your model. Follow the examples in the notebooks.
    * Create docker image:
    > Example code
    ```
    deploy_obj = AMLDeployment(deployment_name=image_name, associated_DNNModel=dnn_model) # dnn_model is the model you trained
    deploy_obj.build_docker_image()
    ```
1. Find your container image information:
    * Find your model management account on the [Azure portal](https://portal.azure.com/). You can find it by searching for your model management account name or you can find it under your resource group. 
    * Go to your model management account link and click "Model Management" under "APPLICATION SETTINGS" section.
    * Find your image under Images and get your **image address** link. You will need this for deloyment in the next section. It will be something similar to this: mlcrpacr************.azurecr.io/imagesimilarity:4. mlcrpacr************ is your <b>Azure Container Registry name</b>.
1. [Get your container registry login information](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-deploy-machine-learning#view-the-container-repository):
    * On the Azure portal, go to All Services and Select Container registries.
    * Select your Registry registry (The name you got from the previous step). The name should start with **mlcr** and it belongs to the resource group, location, and subscription that you used to set up Module Management account.
    * Select Access keys
    * Copy the **Login server**, **Username**, and **Password**. You need these to access the registry from your Edge devices.
1. [Add registry credentials to your Edge device](https://docs.microsoft.com/en-us/azure/iot-edge/tutorial-deploy-machine-learning#add-registry-credentials-to-your-edge-device). You only need to do this once.
    * ```iotedgectl login --address <registry-login-server> --username <registry-username> --password <registry-password>```


## Deploy computer vision container image to IoT Edge device:
1. On the Azure portal, navigate to your IoT hub.
1. Go to IoT Edge (preview) and select your IoT Edge device.
1. Select **Set modules**.
1. Select **Add IoT Edge Module**.
1. In the **Name** field, enter a name, **yourmodulename**. 
1. In the **Image** field, enter your **image address** you found in the previous section. For example: `<registry_name>.azurecr.io/machinelearningmodule:1`.
1. In the Container Create Options field, set the following config. You can change the "HostPort" Binding port number (5001) to your desired **port** number.
    > 
    ```
    {
      "HostConfig": {
        "PortBindings": {
          "5001/tcp": [
            {
              "HostPort": "5001"
            }
          ]
        }
      }
    }
    ```

1. Select **Save**.
1. Back in the Add Modules step, select **Next**.
1. In the Specify Routes step. Put the following: If you need to have advanced routes, like sending data to cloud, sending data between containers, please modify config based on your need.
```
{}
```
1. Select **Next**.
1. In the Review Template step, select **Submit**.
1. Return to the device details page and select **Refresh**. You should see the new **yourmodulename** module running.
1. After it is deployed. Your local **service endpoint** will be "http://localhost:5001/score". The port number **5001** is the one you set in previous step. You can check the running status locally by: `docker ps`

## IoT Edge deployment local scoring
Once the deployment is finished, you can make REST API requests to the **service endpoint** for scoring.
Here is a sample script in Python and C# for scoring. You can construct REST API calls in any programming language by following the schema in the example scripts.

In [ ]:
# Following is an example python script to score a deployed model:

def score_image_with_http(image, service_endpoint_url, service_key=None, parameters={}):
    """Score local image with http request

    Args:
        image (str): Image file path
        service_endpoint_url(str): web service endpoint url
        service_key(str): Service key. None for local deployment.
        parameters (dict): Additional request paramters in dictionary. Default is {}.


    Returns:
        str: serialized result 
    """
    import requests
    from io import BytesIO
    import base64
    import json

    if service_key is None:
        headers = {'Content-Type': 'application/json'}
    else:
        headers = {'Content-Type': 'application/json',
                   "Authorization": ('Bearer ' + service_key)}
    payload = []
    encoded = None
    
    # Read image
    with open(image,'rb') as f:
        image_buffer = BytesIO(f.read()) ## Getting an image file represented as a BytesIO object
        
    # Convert your image to base64 string
    # image_in_base64 : "b'{base64}'"
    encoded = base64.b64encode(image_buffer.getvalue())
    image_request = {"image_in_base64": "{0}".format(encoded), "parameters": parameters}
    payload.append(image_request)
    body = json.dumps(payload)
    r = requests.post(service_endpoint_url, data=body, headers=headers)
    try:
        result = json.loads(r.text)
        json.loads(result[0])
    except:
        raise ValueError("Incorrect output format. Result cant not be parsed: " + r.text)
    return result[0]

# Test with images
image = test_set.images[0].storage_path # A local image file
score_image_with_http(image, service_endpoint_url, service_key=None) # Local scoring the service_key is None

In [ ]:
# Following is an example C# script to score a deployed model:
using Newtonsoft.Json;
using System;
using System.IO;
using System.Net.Http;
using System.Text;

namespace ExampleScoring
{
    class Program
    {
        static void Main()
        {
            const string url = "http://localhost:5001/score";
            const string imgPath = "eval.jpg";

            using (var client = new HttpClient())
            {
                var bytes = File.ReadAllBytes(imgPath);
                var base64 = Convert.ToBase64String(bytes, Base64FormattingOptions.None);
                var data = new[]
                {
                    new
                    {
                        image_in_base64 = $"b'{base64}'",
                        parameters = new { }
                    }
                };
                var json = JsonConvert.SerializeObject(data);
                var content = new StringContent(json, Encoding.UTF8, "application/json");

                var routingId = "";
                client.DefaultRequestHeaders.Add("X-Marathon-App-Id", routingId);

                var response = client.PostAsync(url, content).Result;
                var result = response.Content.ReadAsStringAsync().Result;
                Console.WriteLine($"{response.StatusCode}: {result}");
                Console.ReadLine();
            }
        }
    }
}


## Deployment API:

> **Examples:**
- ```deploy_obj = AMLDeployment(deployment_name=deployment_name, associated_DNNModel=dnn_model, aml_env="cluster")``` # create deployment object
- ```deploy_obj.deploy()``` # deploy web service
- ```deploy_obj.status()``` # get status of deployment
- ```deploy_obj.score_image(local_image_path_or_image_url)``` # score an image
- ```deploy_obj.delete()``` # delete the web service
- ```deploy_obj.build_docker_image()``` # build docker image without creating webservice
- ```AMLDeployment.list_deployment()``` # list existing deployment
- ```AMLDeployment.delete_if_service_exist(deployment_name)``` # delete if the service exists with the deployment name

## API Documentation:

For more API details, please check the API doc. For more advanced operations related to deployment, please check the [model management CLI reference](https://docs.microsoft.com/en-us/azure/machine-learning/preview/model-management-cli-reference).

## Deployment management with web portal:

You can go to [Azure portal](https://ms.portal.azure.com/) to track and manage your deployments. From Azure portal, find your Machine Learning Model Management account page (You can search for your model management account name). Then go to: the model management account page->Model Management->Services.

# 5. Advanced topics and FAQ:<a id='FAQ'></a>
- [Enable SSL on an Azure Machine Learning Compute cluster and score with HTTPS](https://docs.microsoft.com/en-us/azure/machine-learning/desktop-workbench/how-to-setup-ssl-on-mlc)
- [Scale clusters and services](https://docs.microsoft.com/en-us/azure/machine-learning/desktop-workbench/how-to-scale-clusters)
- [Model Management CLI reference](https://docs.microsoft.com/en-us/azure/machine-learning/desktop-workbench/model-management-cli-reference)
- [Load Testing](https://blogs.technet.microsoft.com/machinelearning/2018/05/02/kubernetes-load-testing/)

# 6.Troubleshooting:<a id='troubleshooting'></a>
- Missing Azure CLI package? 
    >If you see some python azure packages are missing when running "az ml .." commands, please run "pip install -r https://aka.ms/az-ml-o16n-cli-requirements-file" in AML Workbench command line prompt to update your CLI packages. This may happen if you upgrade your AML Workbench from an old version.
- Timeout error when building Docker Image? 
    >You can try a model management account with a different region. Region of "eastus2" sometimes has timeout issue.
- Cluster setup fail? 
    >If you cluster setup ends up in "Failed" status, you can try a different region. There is a known issue with region of "westcentralus". 
- Warning error when running CLI setup helper, "WARNING: The behavior of this command has been altered by the following extension: azure-cli-ml"?
    >Please try "pip uninstall azure-cli-ml". This is because your upgraded workbench has duplicate versions of azure-cli-ml package.
- Interactive CLI setup helper doesn't work correctly? 
    >You can try the manual CLI setup option in this notebook.
- [More troubleshooting service deployment and environment setup guide](https://docs.microsoft.com/en-us/azure/machine-learning/desktop-workbench/how-to-deploy-troubleshooting-guide)